# Graph-to-Hypergraph KNN Lifting Tutorial

***
This notebook shows how to import a dataset, with the desired lifting, and how to run a neural network using the loaded data.

The notebook is divided into sections:

- [Loading the dataset](#loading-the-dataset) loads the config files for the data and the desired tranformation, createsa a dataset object and visualizes it.
- [Loading and applying the lifting](#loading-and-applying-the-lifting) defines a simple neural network to test that the lifting creates the expected incidence matrices.
- [Create and run a simplicial nn model](#create-and-run-a-simplicial-nn-model) simply runs a forward pass of the model to check that everything is working as expected.

***
***

Note that for simplicity the notebook is setup to use a simple graph. However, there is a set of available datasets that you can play with.

To switch to one of the available datasets, simply change the *dataset_name* variable in [Dataset config](#dataset-config) to one of the following names:

* cocitation_cora
* cocitation_citeseer
* cocitation_pubmed
* MUTAG
* NCI1
* NCI109
* PROTEINS_TU
* AQSOL
* ZINC
***

### Imports and utilities

In [ ]:
# With this cell any imported module is reloaded before each cell execution
%load_ext autoreload
%autoreload 2
from modules.data.load.loaders import GraphLoader
from modules.data.preprocess.preprocessor import PreProcessor
from modules.utils.utils import (
    describe_data,
    load_dataset_config,
    load_model_config,
    load_transform_config,
)

## Loading the Dataset

Here we just need to spicify the name of the available dataset that we want to load. First, the dataset config is read from the corresponding yaml file (located at `/configs/datasets/` directory), and then the data is loaded via the implemented `Loaders`.


In [ ]:
dataset_name = "manual_dataset"
dataset_config = load_dataset_config(dataset_name)
loader = GraphLoader(dataset_config)

We can then access to the data through the `load()`method:

In [ ]:
dataset = loader.load()
describe_data(dataset)

## Loading and Applying the Lifting

In this section, we will instantiate the lifting we want to apply to the data. In this tutorial, we apply Kernel Lifting. A kernel over a graph is a positive semidefinite matrix describing the node similarity. There were plenty of works on graph kernels [1-4]. Our approach is straightforward: (1) calculate a kernel over the nodes of a graph, (2) calculate a kernel over the features of the graph, (3) combine them, and (4) filter out `fraction` of the closest nodes, (5) if vertices are connected to a given node --- add them as a hyperedge.

Let us expand on each step:
- (1) Graph kernel can be computed either as Mat\'ern or heat kernel,
- (2) A wide range of kernels can be used to calculate the kernel over features, such as RBF or periodic. In order to "turn off" features, use `identity` as a kernel
- (3) Kernels can be combined, for instance, via Hadamard product or sum.
- (4) We select `fraction`-quantile of values in the kernel to determine the similarity threshold, then we filter out values lower than this threshold and obtain new connectivity based on the kernel,
- (5) If vertices remain connected to a given vertex in this new graph, we construct a hyperedge between them.

The kernels in the proposed method are the following:
$$K(v_i, v_j) = C(K_v(v_i, v_j), K_x(x_i, x_j)),$$
Where $K_v$ is a kernel over the nodes of the graph, $K_x$ is a kernel over the features of the graph, and $C$ is a combining function (for instance, Hadamard product or sum). The resulting kernel defines a similarity measure over the graph, and we use this similarity to construct the hyperedges (`1-fraction`-quantile of the closest similarities is used).

It is worth noting that by using identity $K_v = I$ and $C(K_1, K_2) = K_1 \odot K_2$, the lifting utilizes only features; similarly, it can be used to use only graph structure.


***
[[1]](http://people.cs.uchicago.edu/~risi/papers/diffusion-kernels.pdf) Kondor, R. I., & Lafferty, J. (2002, July). Diffusion kernels on graphs and other discrete structures. In Proceedings of the 19th international conference on machine learning (Vol. 2002, pp. 315-322).  
[[2]](https://arxiv.org/abs/2010.15538) Borovitskiy, V., Azangulov, I., Terenin, A., Mostowsky, P., Deisenroth, M., & Durrande, N. (2021, March). Matérn Gaussian processes on graphs. In International Conference on Artificial Intelligence and Statistics (pp. 2593-2601). PMLR.  
[[3]](https://arxiv.org/abs/2010.15538) Nikitin, A. V., John, S. T., Solin, A., & Kaski, S. (2022, May). Non-separable spatio-temporal graph kernels via SPDEs. In International Conference on Artificial Intelligence and Statistics (pp. 10640-10660). PMLR.  
[[4]](https://aaltodoc.aalto.fi/server/api/core/bitstreams/f206b46d-0b64-4b27-93e1-bd4f4a0f5bd0/content) Nikitin, A. (2024). Probabilistic Methods for Predictive Maintenance and Beyond: Graph and Human-in-the-Loop Machine Learning.  
***

For hypergraphs, creating a lifting involves creating the `incidence_hyperedges` matrix.

Similarly to before, we can specify the transformation we want to apply through its type and id --the corresponding config files located at `/configs/transforms.` 

Note that the *tranform_config* dictionary generated below can contain a sequence of transforms if it is needed.

This can also be used to explore liftings from one topological domain to another; for example, using two liftings, it is possible to achieve a sequence such as graph -> simplicial complex -> hypergraph. 

In [ ]:
# Define transformation type and id
transform_type = "liftings"
# If the transform is a topological lifting, it should include both the type of the lifting and the identifier
transform_id = "graph2hypergraph/kernel_lifting"

# Read yaml file
transform_config = {
    "lifting": load_transform_config(transform_type, transform_id)
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}

We than apply the transform via our `PreProcessor`:

In [ ]:
lifted_dataset = PreProcessor(dataset, transform_config, loader.data_dir)
describe_data(lifted_dataset)

## Create and Run a Simplicial NN Model

In this section a simple model is created to test that the used lifting works as intended. In this case the model uses the `incidence_hyperedges` matrix so the lifting should make sure to add it to the data.

In [ ]:
from modules.models.hypergraph.unigcn import UniGCNModel

model_type = "hypergraph"
model_id = "unigcn"
model_config = load_model_config(model_type, model_id)

model = UniGCNModel(model_config, dataset_config)

In [ ]:
y_hat = model(lifted_dataset.get(0))

If everything is correct the cell above should execute without errors. 